In [11]:
pip install google-generativeai pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 10.8 MB/s  0:00:01eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 10.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [30]:
import os
import json
import re
import google.generativeai as genai

In [31]:
GEMINI_API_KEY= ""

In [32]:
genai.configure(api_key=GEMINI_API_KEY)

In [34]:
import pandas as pd
df_examples = pd.read_csv("dataset.csv")
df_examples.head(2)


# Assume your Excel has columns: "message" and "label"
examples = []
for _, row in df_examples.iterrows():
    msg = row['message_text']
    lbl = row['label']  # 'Scam', 'Not Scam', or 'Uncertain'
    reasoning = row.get('reasoning', f"Classified as {lbl}")  # optional column
    intent = row.get('intent_type', "Unknown")  # optional column
    risk_factors = row.get('flag_reason', "[]")  # optional column

    # Format as JSON string
    example_json = f"""{{
  "label": "{lbl}",
  "reasoning": "{reasoning}",
  "intent": "{intent}",
  "risk_factors": {risk_factors}
}}"""

    examples.append(f'Input message: "{msg}"\nResponse:\n{example_json}')

,message_text,label,intent_type,flag_reason
0,Reminder: Your gas cylinder booking is confirmed.,Not Scam,Service Reminder,Legitimate communication
1,Power cut scheduled from 2 PM to 4 PM tomorrow.,Not Scam,Informational Alert,Legitimate communication


In [17]:
SCAM_GUARD_SYSTEM_PROMPT = """
You are "Scam Guard" — a smart AI that helps users identify potential scams in messages. 
Analyze each message carefully and provide structured JSON output with reasoning.

Rules:
1. RESPONSE FORMAT (STRICT JSON):
{{
  "label": "Scam" | "Not Scam" | "Uncertain",
  "reasoning": "Step-by-step reasoning why the message is suspicious or safe",
  "intent": "Purpose of the message",
  "risk_factors": ["list", "of", "red flags"]
}}

2. TONE:
- Friendly but cautionary
- Informative, do not over-alarm

3. EXAMPLES:
""" + "\n\n".join(examples)

In [18]:
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=SCAM_GUARD_SYSTEM_PROMPT
)

chat = model.start_chat()      

In [19]:
   def safe_parse_json(text: str):
    """Extract first JSON object-looking substring and parse."""
    stack = []
    start = None
    for i, ch in enumerate(text):
        if ch == '{':
            if start is None:
                start = i
            stack.append('{')
        elif ch == '}':
            if stack:
                stack.pop()
                if not stack and start is not None:
                    candidate = text[start:i+1]
                    try:
                        return json.loads(candidate)
                    except Exception:
                        candidate_fixed = candidate.replace("'", "\"")
                        candidate_fixed = re.sub(r",\s*}", "}", candidate_fixed)
                        candidate_fixed = re.sub(r",\s*]", "]", candidate_fixed)
                        try:
                            return json.loads(candidate_fixed)
                        except Exception:
                            pass
                    start = None
    raise ValueError("No valid JSON object found in model output.")


def detect_scam(message: str):
    """Send message to Scam Guard AI and return JSON output."""
    raw_output = chat.send_message(message).text
    try:
        return safe_parse_json(raw_output)
    except Exception:
        # fallback if JSON parsing fails
        return {
            "label": "Uncertain",
            "reasoning": "Failed to parse model output as JSON.",
            "intent": "Unknown",
            "risk_factors": []
        }


def main():
    print("Scam Guard AI - Type 'exit' to quit.\n")
    while True:
        msg = input("Enter message: ").strip()
        if msg.lower() == "exit":
            print("Goodbye! Stay safe online. 🛡️")
            break
        if not msg:
            continue

        result = detect_scam(msg)
        print("\n=== Detection Result ===")
        print(json.dumps(result, indent=2))
        print("=======================\n")


if __name__ == "__main__":
    main()

Scam Guard AI - Type 'exit' to quit.



Enter message:  exit


Goodbye! Stay safe online. 🛡️
